In [4]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
  #### imort libraries ####
import pandas as pd 
from pandas.core.frame import DataFrame 
import re #regular expretion    
import nltk  # natural language processing
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *
  #sentement analysis 
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import string , spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS


from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
reader=pd.read_csv('/content/gdrive/MyDrive/Bitcoin_tweets.csv')
reader

,Unnamed: 0,date,Open_x,High_x,Low_x,Close_x,Volume_x,Unnamed: 0.1,Open_y,High_y,Low_y,Close_y,Volume_y,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,text,hashtags,source,is_retweet
0,87838,2021-02-05 10:53:00,37394.17,37401.05,37354.09,37389.76,41,87838.0,37394.17,37401.05,37354.09,37389.76,41.0,Nick Doevendans,"Edam-Volendam, Nederland","Amateur historicus m.n. WW2, schrijver, muziek...",2020-06-12 16:50:07,37.0,123.0,410.0,False,#reddcoin #rdd @reddcoin to the moon #altcoin ...,"['reddcoin', 'rdd', 'altcoin', 'turnreddcoinin...",Twitter for iPhone,False
1,87839,2021-02-05 10:54:00,37389.76,37416.07,37379.86,37380.00,28,87839.0,37389.76,37416.07,37379.86,37380.00,28.0,Nick Doevendans,"Edam-Volendam, Nederland","Amateur historicus m.n. WW2, schrijver, muziek...",2020-06-12 16:50:07,37.0,123.0,410.0,False,#reddcoin #rdd @reddcoin to the moon #altcoin ...,"['reddcoin', 'rdd', 'altcoin', 'turnreddcoinin...",Twitter for iPhone,False
2,87840,2021-02-05 10:55:00,37380.01,37436.53,37380.00,37425.34,21,87840.0,37380.01,37436.53,37380.00,37425.34,21.0,Lion Period with MR.Emre,"Los Angeles, CA",A Lion Story (Private group closed)\n#CryptoMo...,2020-11-22 16:06:54,69.0,52.0,191.0,False,$PERL 0.06.\nI have insisted that since 0.02 i...,"['bitcoin', 'btc']",Twitter for Android,False
3,87841,2021-02-05 10:56:00,37425.35,37438.41,37395.91,37436.50,26,87841.0,37425.35,37438.41,37395.91,37436.50,26.0,Lion Period with MR.Emre,"Los Angeles, CA",A Lion Story (Private group closed)\n#CryptoMo...,2020-11-22 16:06:54,69.0,52.0,191.0,False,$PERL 0.06.\nI have insisted that since 0.02 i...,"['bitcoin', 'btc']",Twitter for Android,False
4,87842,2021-02-05 10:57:00,37436.51,37450.00,37420.00,37436.26,29,87842.0,37436.51,37450.00,37420.00,37436.26,29.0,Lion Period with MR.Emre,"Los Angeles, CA",A Lion Story (Private group closed)\n#CryptoMo...,2020-11-22 16:06:54,69.0,52.0,191.0,False,$PERL 0.06.\nI have insisted that since 0.02 i...,"['bitcoin', 'btc']",Twitter for Android,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436482,524320,2021-12-06 06:08:00,48734.90,48750.00,48721.57,48721.57,10,524320.0,48734.90,48750.00,48721.57,48721.57,10.0,Raj Patel,"Artesia, CA",Life is too short for bad vibes!,2012-01-28 23:06:05,1.0,6.0,9.0,False,Who thinks #Bitcoin investment is risky 🤔,['Bitcoin'],Twitter for Android,False
436483,524321,2021-12-06 06:09:00,48721.57,48753.55,48717.41,48738.11,14,524321.0,48721.57,48753.55,48717.41,48738.11,14.0,Raj Patel,"Artesia, CA",Life is too short for bad vibes!,2012-01-28 23:06:05,1.0,6.0,9.0,False,Who thinks #Bitcoin investment is risky 🤔,['Bitcoin'],Twitter for Android,False
436484,524322,2021-12-06 06:10:00,48738.12,48752.31,48700.01,48708.85,12,524322.0,48738.12,48752.31,48700.01,48708.85,12.0,Raj Patel,"Artesia, CA",Life is too short for bad vibes!,2012-01-28 23:06:05,1.0,6.0,9.0,False,Who thinks #Bitcoin investment is risky 🤔,['Bitcoin'],Twitter for Android,False
436485,524323,2021-12-06 06:11:00,48708.84,48712.20,48660.74,48696.41,11,524323.0,48708.84,48712.20,48660.74,48696.41,11.0,Raj Patel,"Artesia, CA",Life is too short for bad vibes!,2012-01-28 23:06:05,1.0,6.0,9.0,False,Who thinks #Bitcoin investment is risky 🤔,['Bitcoin'],Twitter for Android,False


In [7]:
btc=reader.loc[:,["date","text","user_location","user_description","user_followers","user_friends"]]
btc


,date,text,user_location,user_description,user_followers,user_friends
0,2021-02-05 10:53:00,#reddcoin #rdd @reddcoin to the moon #altcoin ...,"Edam-Volendam, Nederland","Amateur historicus m.n. WW2, schrijver, muziek...",37.0,123.0
1,2021-02-05 10:54:00,#reddcoin #rdd @reddcoin to the moon #altcoin ...,"Edam-Volendam, Nederland","Amateur historicus m.n. WW2, schrijver, muziek...",37.0,123.0
2,2021-02-05 10:55:00,$PERL 0.06.\nI have insisted that since 0.02 i...,"Los Angeles, CA",A Lion Story (Private group closed)\n#CryptoMo...,69.0,52.0
3,2021-02-05 10:56:00,$PERL 0.06.\nI have insisted that since 0.02 i...,"Los Angeles, CA",A Lion Story (Private group closed)\n#CryptoMo...,69.0,52.0
4,2021-02-05 10:57:00,$PERL 0.06.\nI have insisted that since 0.02 i...,"Los Angeles, CA",A Lion Story (Private group closed)\n#CryptoMo...,69.0,52.0
...,...,...,...,...,...,...
436482,2021-12-06 06:08:00,Who thinks #Bitcoin investment is risky 🤔,"Artesia, CA",Life is too short for bad vibes!,1.0,6.0
436483,2021-12-06 06:09:00,Who thinks #Bitcoin investment is risky 🤔,"Artesia, CA",Life is too short for bad vibes!,1.0,6.0
436484,2021-12-06 06:10:00,Who thinks #Bitcoin investment is risky 🤔,"Artesia, CA",Life is too short for bad vibes!,1.0,6.0
436485,2021-12-06 06:11:00,Who thinks #Bitcoin investment is risky 🤔,"Artesia, CA",Life is too short for bad vibes!,1.0,6.0


In [8]:
#from sklearn.model_selection import train_test_split 
#X_train, X_test, y_train, y_test = train_test_split(reader, train_size=250000, test_size=20000)
import pandas  as pd 
df=pd.DataFrame(reader)
df.sample(n=300000)


,Unnamed: 0,date,Open_x,High_x,Low_x,Close_x,Volume_x,Unnamed: 0.1,Open_y,High_y,Low_y,Close_y,Volume_y,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,text,hashtags,source,is_retweet
377577,465415,2021-10-26 08:23:00,62921.79,62974.36,62919.55,62973.03,26,465415.0,62921.79,62974.36,62919.55,62973.03,26.0,Certified Luffy Boy,Lambo Land,Syracuse University MBA ‘21 x USMA ‘14. I dabb...,2021-02-04 12:29:39,153.0,348.0,3373.0,False,$nezuko look at this bullish chart \n@nezukoin...,"['nezuko', 'luffy', 'goku', 'eth', 'btc']",Twitter for iPhone,False
107824,195662,2021-04-21 13:16:00,55387.97,55494.97,55339.99,55455.93,93,195662.0,55387.97,55494.97,55339.99,55455.93,93.0,Crypto: The Future Of Money,United States,“Do not store up for yourselves treasures on e...,2010-01-22 02:54:29,25870.0,5.0,1078.0,False,Vantagepoint A.I. Adds Cardano (ADA) and Other...,"['digibyte', 'dgb', 'altcoins', 'coins', 'etf'...",Twitter Web App,False
267767,355605,2021-08-10 19:43:00,45485.72,45513.33,45483.95,45499.54,22,355605.0,45485.72,45513.33,45483.95,45499.54,22.0,KyleFromOhio ᕫ 📈,South Carolina,Tried to mine #bitcoin in 2011 on a laptop.\nM...,2013-04-04 03:27:13,1143.0,223.0,42177.0,False,#bitcoin just closed above the 200D and 50D MA...,['bitcoin'],TweetDeck,False
223636,311474,2021-07-11 04:12:00,33346.84,33361.62,33333.33,33350.29,31,311474.0,33346.84,33361.62,33333.33,33350.29,31.0,Cardano News Feed,Alonzo,#cardano #ADA #lovelace #bitcoin #cardanocommu...,2021-06-16 11:55:44,10.0,118.0,4.0,False,Why ‘Cardano is just teasing us’\n\n💡 https://...,"['cardano', 'ADA', 'lovelace', 'bitcoin', 'car...",Cardanona Twitter App,False
379305,467143,2021-10-27 13:11:00,59040.55,59083.64,59023.87,59063.94,27,467143.0,59040.55,59083.64,59023.87,59063.94,27.0,jiyaang,World,to daa moon,2021-06-29 06:52:40,48.0,102.0,144.0,False,@portal_finance @joinrepublic I am so sorry ex...,"['PortalFinance', 'bitcoin', 'DeFi']",Twitter for Android,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73934,161772,2021-03-28 21:56:00,55202.70,55248.48,55200.00,55208.18,20,161772.0,55202.70,55248.48,55200.00,55208.18,20.0,CryptoInvest.Tech,Worldwide,https://t.co/gDTgo1OZVm provides you the Best ...,2019-04-13 19:28:05,407.0,8.0,21548.0,False,Altcoins sell-off after Bitcoin price rejects ...,['Bitcoin'],Twitter for Android,False
272814,360652,2021-08-14 12:20:00,46294.41,46358.66,46288.34,46348.68,22,360652.0,46294.41,46358.66,46288.34,46348.68,22.0,KyleFromOhio ᕫ 📈,South Carolina,Tried to mine #bitcoin in 2011 on a laptop.\nM...,2013-04-04 03:27:13,1143.0,223.0,42177.0,False,#bitcoin just closed above the 200D and 50D MA...,['bitcoin'],TweetDeck,False
297769,385607,2021-08-31 20:15:00,47270.32,47281.84,47234.56,47280.12,41,385607.0,47270.32,47281.84,47234.56,47280.12,41.0,Bitcoin LightningNetwork+ News ⚡️,Planet HODL,"#Bitcoin #LightningNetwork: Instant, Cheap, Co...",2013-11-17 23:36:18,10270.0,5970.0,3120.0,False,Demonstration of K1 Bitcoin ATM. First #Bitcoi...,"['Bitcoin', 'Lightning']",Twitter for iPhone,False
422822,510660,2021-11-26 18:28:00,53986.47,54053.78,53986.46,54000.37,39,510660.0,53986.47,54053.78,53986.46,54000.37,39.0,Raj Patel,"Artesia, CA",Life is too short for bad vibes!,2012-01-28 23:06:05,1.0,6.0,9.0,False,Who thinks #Bitcoin investment is risky 🤔,['Bitcoin'],Twitter for Android,False


In [9]:
reader.shape

(436487, 25)

In [10]:
############ \Drop Emoji from tweets ###########
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+", flags = re.UNICODE)
    #print(regrex_pattern)
    return regrex_pattern.sub(r'', text)
    


reader['text'] = reader['text'].apply(deEmojify) 
reader

,Unnamed: 0,date,Open_x,High_x,Low_x,Close_x,Volume_x,Unnamed: 0.1,Open_y,High_y,Low_y,Close_y,Volume_y,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,text,hashtags,source,is_retweet
0,87838,2021-02-05 10:53:00,37394.17,37401.05,37354.09,37389.76,41,87838.0,37394.17,37401.05,37354.09,37389.76,41.0,Nick Doevendans,"Edam-Volendam, Nederland","Amateur historicus m.n. WW2, schrijver, muziek...",2020-06-12 16:50:07,37.0,123.0,410.0,False,#reddcoin #rdd @reddcoin to the moon #altcoin ...,"['reddcoin', 'rdd', 'altcoin', 'turnreddcoinin...",Twitter for iPhone,False
1,87839,2021-02-05 10:54:00,37389.76,37416.07,37379.86,37380.00,28,87839.0,37389.76,37416.07,37379.86,37380.00,28.0,Nick Doevendans,"Edam-Volendam, Nederland","Amateur historicus m.n. WW2, schrijver, muziek...",2020-06-12 16:50:07,37.0,123.0,410.0,False,#reddcoin #rdd @reddcoin to the moon #altcoin ...,"['reddcoin', 'rdd', 'altcoin', 'turnreddcoinin...",Twitter for iPhone,False
2,87840,2021-02-05 10:55:00,37380.01,37436.53,37380.00,37425.34,21,87840.0,37380.01,37436.53,37380.00,37425.34,21.0,Lion Period with MR.Emre,"Los Angeles, CA",A Lion Story (Private group closed)\n#CryptoMo...,2020-11-22 16:06:54,69.0,52.0,191.0,False,$PERL 0.06.\nI have insisted that since 0.02 i...,"['bitcoin', 'btc']",Twitter for Android,False
3,87841,2021-02-05 10:56:00,37425.35,37438.41,37395.91,37436.50,26,87841.0,37425.35,37438.41,37395.91,37436.50,26.0,Lion Period with MR.Emre,"Los Angeles, CA",A Lion Story (Private group closed)\n#CryptoMo...,2020-11-22 16:06:54,69.0,52.0,191.0,False,$PERL 0.06.\nI have insisted that since 0.02 i...,"['bitcoin', 'btc']",Twitter for Android,False
4,87842,2021-02-05 10:57:00,37436.51,37450.00,37420.00,37436.26,29,87842.0,37436.51,37450.00,37420.00,37436.26,29.0,Lion Period with MR.Emre,"Los Angeles, CA",A Lion Story (Private group closed)\n#CryptoMo...,2020-11-22 16:06:54,69.0,52.0,191.0,False,$PERL 0.06.\nI have insisted that since 0.02 i...,"['bitcoin', 'btc']",Twitter for Android,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436482,524320,2021-12-06 06:08:00,48734.90,48750.00,48721.57,48721.57,10,524320.0,48734.90,48750.00,48721.57,48721.57,10.0,Raj Patel,"Artesia, CA",Life is too short for bad vibes!,2012-01-28 23:06:05,1.0,6.0,9.0,False,Who thinks #Bitcoin investment is risky,['Bitcoin'],Twitter for Android,False
436483,524321,2021-12-06 06:09:00,48721.57,48753.55,48717.41,48738.11,14,524321.0,48721.57,48753.55,48717.41,48738.11,14.0,Raj Patel,"Artesia, CA",Life is too short for bad vibes!,2012-01-28 23:06:05,1.0,6.0,9.0,False,Who thinks #Bitcoin investment is risky,['Bitcoin'],Twitter for Android,False
436484,524322,2021-12-06 06:10:00,48738.12,48752.31,48700.01,48708.85,12,524322.0,48738.12,48752.31,48700.01,48708.85,12.0,Raj Patel,"Artesia, CA",Life is too short for bad vibes!,2012-01-28 23:06:05,1.0,6.0,9.0,False,Who thinks #Bitcoin investment is risky,['Bitcoin'],Twitter for Android,False
436485,524323,2021-12-06 06:11:00,48708.84,48712.20,48660.74,48696.41,11,524323.0,48708.84,48712.20,48660.74,48696.41,11.0,Raj Patel,"Artesia, CA",Life is too short for bad vibes!,2012-01-28 23:06:05,1.0,6.0,9.0,False,Who thinks #Bitcoin investment is risky,['Bitcoin'],Twitter for Android,False


In [11]:
reader.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436487 entries, 0 to 436486
Data columns (total 25 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        436487 non-null  int64  
 1   date              436487 non-null  object 
 2   Open_x            436487 non-null  float64
 3   High_x            436487 non-null  float64
 4   Low_x             436487 non-null  float64
 5   Close_x           436487 non-null  float64
 6   Volume_x          436487 non-null  int64  
 7   Unnamed: 0.1      436487 non-null  float64
 8   Open_y            436487 non-null  float64
 9   High_y            436487 non-null  float64
 10  Low_y             436487 non-null  float64
 11  Close_y           436487 non-null  float64
 12  Volume_y          436487 non-null  float64
 13  user_name         436487 non-null  object 
 14  user_location     436487 non-null  object 
 15  user_description  436487 non-null  object 
 16  user_created      43

In [12]:
features = reader.iloc[:, 21].values## text 
labels = reader.iloc[:, 5].values ## y : 1 

In [13]:
labels

array([37389.76, 37380.  , 37425.34, ..., 48708.85, 48696.41, 48656.86])

In [14]:
import nltk 
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
processed_features = []

for sentence in range(0, len(features)):
    # Remove all the special characters
    processed_feature = re.sub(r'\W', ' ', str(features[sentence]))
    #remove rul: 
    processed_feature = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',processed_feature)
    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    processed_feature = re.sub(r'#([^\s]+)', r'\1', processed_feature)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()
    # Lemmatization
    processed_feature = processed_feature.split()
    #processed_feature = [ word for word in processed_feature if word not in stop_words and word not in punctuations ]
 
    processed_feature = [stemmer.lemmatize(word) for word in processed_feature]
    processed_feature = ' '.join(processed_feature)
    
    processed_features.append(processed_feature)

In [16]:
type(processed_feature)

str

"\ndef remove_stop_words(processed_features):\n    removed_stop_words = []\n    for review in processed_features:\n        removed_stop_words.append(\n            ' '.join([word for word in review.split()  if word not in stop_words]))\n    return removed_stop_words\n\n    "

In [ ]:
#remove_stop_words(processed_features)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer (max_features=200, min_df=7, max_df=0.8, stop_words=stopwords.words('english'))
processed_features = vectorizer.fit_transform(processed_features).toarray()

In [18]:
processed_features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
processed_features.shape

(436487, 200)

In [16]:
pd.DataFrame(processed_features)[0]

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
436482    0.0
436483    0.0
436484    0.0
436485    0.0
436486    0.0
Name: 0, Length: 436487, dtype: float64

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.2, random_state=0)

#Ridge : 

In [21]:
# Alpha by default is 1.0
from sklearn.linear_model import Ridge
ridge = Ridge()
ridge.fit(X_train, y_train)
print('Train score: {}'.format(ridge.score(X_train, y_train)))
print('Test score: {}'.format(ridge.score(X_test,y_test)))

Train score: 0.5651898547206885
Test score: 0.5637648293277299


In [19]:
# Use GridSearchCV to find the best alpha
from sklearn.model_selection import GridSearchCV

ridge = Ridge()
param_grid = {'alpha':[0.001,0.01, 0.1,1]}

grid_search = GridSearchCV(ridge, param_grid, cv = 5)
grid_search.fit(X_train, y_train)

#use gridsearch to get the best parameters, alpha=0.001
print('Best parameters{}'.format(grid_search.best_params_))
print('Best score {:.2f}'.format(grid_search.best_score_))

Best parameters{'alpha': 0.1}
Best score 0.56


#LASSO

In [23]:
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
lasso = Lasso(max_iter = 100000)
lasso.fit(X_train, y_train)
print('Train score: {}'.format(lasso.score(X_train, y_train)))
print('Test score: {}'.format(lasso.score(X_test,y_test)))
print('Number of features used: {}'.format(np.sum(lasso.coef_ != 0)))

#### ROWS : 300000  

Train score: 0.5634110985588219
Test score: 0.5621997440818722
Number of features used: 114


In [ ]:
# Use GridSearchCV to find the best alpha
from sklearn.model_selection import GridSearchCV
lasso = Lasso(max_iter = 100000)
param_grid = {'alpha':[0.017, 0.1, 0.01, 0.026 , 1]}

grid_search = GridSearchCV(lasso, param_grid, cv = 5)
grid_search.fit(X_train, y_train)

print('Best parameters{}'.format(grid_search.best_params_))
print('Best score {}'.format(grid_search.best_score_))

# SVR WITH LINEAR - NON LINEAR 

#Linear Support Vector Regression.

Similar to SVR with parameter kernel=’linear’, but implemented in terms of liblinear rather than libsvm, so it has more flexibility in the choice of penalties and loss functions and should scale better to large numbers of samples.

This class supports both dense and sparse input.


In [22]:
from sklearn.svm import LinearSVR
lsvr = LinearSVR()
lsvr.fit(X_train, y_train)
print('Train score: {}'.format(lsvr.score(X_train, y_train)))
print('Test score: {}\n'.format(lsvr.score(X_test,y_test)))

Train score: 0.4146328960975406
Test score: 0.41194896273022374



In [ ]:
############################################################Execution failed 
#from sklearn.svm import SVR
#svr = SVR(kernel='linear')
#svr.fit(X_train, y_train)
#print('Train score: {}'.format(svr.score(X_train, y_train)))
#print('Test score: {}\n'.format(svr.score(X_test,y_test)))